In [ ]:
import cv2
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
CROP_DIM = (0, 40, 640, 360)
NEW_SIZE = (200, 100)

def op_flow_raw(img0, img1):
    gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(
        gray0, gray1, None, 0.5, 3, 5, 3, 5, 1.1, 0
    )
    return flow

def op_flow_bgr(flow, shape):
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mask = np.zeros(shape, dtype=np.uint8)
    mask[..., 0] = ang * 180 / np.pi / 2    # direction
    mask[..., 1] = 255                      # full saturation
    mask[..., 2] = cv2.normalize(           # intensity
        mag, None, 0, 255, cv2.NORM_MINMAX
    )
    bgr = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    return bgr

def format_frame(frame):
    img = Image.fromarray(frame).crop(CROP_DIM).resize(NEW_SIZE)
    return np.array(img)

def split_video(video):
    frames, op_flows, op_bgrs = [], [], []
    cap = cv2.VideoCapture(video)
    pbar = tqdm(total=cap.get(cv2.CAP_PROP_FRAME_COUNT)-1, position=0, leave=2)
    ret, frame1 = cap.read()
    frame1 = format_frame(frame1)
    while ret:
        ret, frame2 = cap.read()
        if ret:
            frame2 = format_frame(frame2)
            frames.append(frame2)
            op_flow = op_flow_raw(frame1, frame2)
            op_flows.append(op_flow)
            op_bgr = op_flow_bgr(op_flow, frame2.shape)
            op_bgrs.append(op_bgr)
            frame1 = frame2
            pbar.update()
        else:
            print('Finished saving '+video)
    return np.array(frames), np.array(op_flows), np.array(op_bgrs)

In [ ]:
lbl = np.loadtxt('./data/train.txt')
with h5py.File('./data/train.h5', 'w') as f:
    frm, opf, opv = split_video('./data/train.mp4')
    f.create_dataset('FRM', frm.shape, data=frm)
    f.create_dataset('OPF', opf.shape, data=opf)
    f.create_dataset('OPV', opv.shape, data=opv)
    f.create_dataset('LBL', lbl.shape, data=lbl)
f.close()

In [ ]:
with h5py.File('./data/train.h5', 'r') as f:
    frm = f['FRM']
    opv = f['OPV']
    for a, b in zip(frm, opv):
        cv2.imshow('FRM', a)
        cv2.imshow('OPV', b)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
f.close()

In [ ]:
class SpeedDataset(Dataset):
    def __init__(self, filename, transform=None):
        super(SpeedDataset, self).__init__()

        self.file = h5py.File(filename)
        self.frm = self.file['FRM']
        self.opf = self.file['OPF']
        self.opv = self.file['OPV']
        self.lbl = self.file['LBL']
        self.transform = transform

    def __len__(self):
        return len(self.frm)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # from h5py
        X_frm = self.frm[idx]
        X_opf = self.opf[idx]
        X_opv = self.opv[idx]
        y = self.lbl[idx+1]

        # apply transform
        if self.transform is not None:
            X_frm = self.transform(X_frm).permute(1, 2, 0)
            X_opf = self.transform(X_opf).permute(1, 2, 0)
            X_opv = self.transform(X_opv).permute(1, 2, 0)

        return X_frm, X_opf, X_opv, y

dataset = SpeedDataset(
    './data/train.h5',
    transforms.Compose([
        transforms.ToTensor()
    ])
)

dataloader = DataLoader(dataset, 100)

In [ ]:
class Conv2DReLU(nn.Module):

    def __init__(
        self, in_channels, out_channels, kernel_size, stride, padding=0
    ):
        super(Conv2DReLU, self).__init__()

        self.conv = Conv2DReLU(
            in_channels, out_channels,
            kernel_size=kernel_size, stride=stride, padding=padding
        )

    def forward(self, x):
        x = F.relu(self.conv(x))
        return x

class Stem(nn.Module):

    def __init__(self, in_channels):
        super(Stem, self).__init__()

        self.base = nn.Sequential(
            Conv2DReLU(in_channels, 32, kernel_size=3, stride=2),
            Conv2DReLU(32, 32, kernel_size=3, stride=1),
            Conv2DReLU(32, 64, kernel_size=3, stride=1, padding=1)
        )
        self.conv0 = Conv2DReLU(64, 96, kernel_size=3, stride=2)
        self.pool0 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.branch0 = nn.Sequential(
            Conv2DReLU(160, 64, kernel_size=1, stride=1),
            Conv2DReLU(64, 96, kernel_size=3, stride=1)
        )
        self.branch1 = nn.Sequential(
            Conv2DReLU(160, 64, kernel_size=1, stride=1),
            Conv2DReLU(64, 64, kernel_size=(7, 1), stride=1, padding=(3, 0)),
            Conv2DReLU(64, 64, kernel_size=(1, 7), stride=1, padding=(0, 3)),
            Conv2DReLU(64, 96, kernel_size=3, stride=1)
        )
        self.conv1 = Conv2DReLU(192, 192, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        x = self.base(x)
        x0 = self.conv0(x)
        x1 = self.pool0(x)
        x = torch.cat((x0, x1), 1)
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x = torch.cat((x0, x1), 1)
        x0 = self.conv1(x)
        x1 = self.pool1(x)
        x = torch.cat((x0, x1), 1)
        return x

class InceptionA:

    def __init__(self):
        super(InceptionA, self).__init__()

        self.branch0 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3),
            Conv2DReLU(384, 96, kernel_size=1, stride=1)
        )
        self.branch1 = nn.Sequential(
            Conv2DReLU(384, 96, kernel_size=1, stride=1)
        )
        self.branch2 = nn.Sequential(
            Conv2DReLU(384, 64, kernel_size=1, stride=1),
            Conv2DReLU(64, 96, kernel_size=3, stride=1)
        )
        self.branch3 = nn.Sequential(
            Conv2DReLU(384, 64, kernel_size=1, stride=1),
            Conv2DReLU(64, 96, kernel_size=3, stride=1),
            Conv2DReLU(96, 96, kernel_size=3, stride=1)
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x = torch.cat((x0, x1, x2, x3), 1)
        return x

class InceptionB:

    def __init__(self):
        super(InceptionB, self).__init__()

        self.branch0 = nn.Sequential(
            nn.AvgPool2d(kernel_size=1)
        )